#Setup and Data Loading

In [ ]:
!pip install xarray netCDF4 dask

import glob

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import xarray as xr
from google.colab import drive
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import (
    Add,
    BatchNormalization,
    Conv2D,
    Cropping2D,
    Input,
    Lambda,
    MaxPooling2D,
    UpSampling2D,
    concatenate,
)
from tensorflow.keras.models import Model

In [ ]:
drive.mount("/content/drive")

file_path_X = "/content/drive/MyDrive/AI_project/without_reduced/5step/"
file_path_Y = "/content/drive/MyDrive/AI_project/with_reduced/5step/"

output_directory = "/content/drive/MyDrive/dataset/output"

files_X = sorted(glob.glob(file_path_X + "WAVEAN20*.nc"))
files_Y = sorted(glob.glob(file_path_Y + "WAVEAN20*.nc"))

ds_features = xr.open_mfdataset(files_X, chunks="auto", engine="netcdf4")
ds_labels = xr.open_mfdataset(files_Y, chunks="auto", engine="netcdf4")

Features:

    VHM0_uncorrected
    WSPD
    WDIR_cos
    WDIR_sin
    VMDR_cos
    VMDR_sin
    month_cos
    month_sin
    hour_cos
    hour_sin

Labels:

    VHM0_groundtruth


In [ ]:
print(ds_features.data_vars)
print(ds_labels.data_vars)

ds_features["WDIR_cos"] = np.cos(np.deg2rad(ds_features["WDIR"]))
ds_features["WDIR_sin"] = np.sin(np.deg2rad(ds_features["WDIR"]))
ds_features["VMDR_cos"] = np.cos(np.deg2rad(ds_features["VMDR"]))
ds_features["VMDR_sin"] = np.sin(np.deg2rad(ds_features["VMDR"]))


ds_features["month_cos"] = np.cos(2 * np.pi * (ds_features.time.dt.month - 1) / 12)
ds_features["month_sin"] = np.sin(2 * np.pi * (ds_features.time.dt.month - 1) / 12)
ds_features["hour_cos"] = np.cos(2 * np.pi * ds_features.time.dt.hour / 24)
ds_features["hour_sin"] = np.sin(2 * np.pi * ds_features.time.dt.hour / 24)


ds_features["month_cos"] = ds_features["month_cos"].broadcast_like(ds_features["VHM0"])
ds_features["month_sin"] = ds_features["month_sin"].broadcast_like(ds_features["VHM0"])
ds_features["hour_cos"] = ds_features["hour_cos"].broadcast_like(ds_features["VHM0"])
ds_features["hour_sin"] = ds_features["hour_sin"].broadcast_like(ds_features["VHM0"])


labels = ds_labels["VHM0"]  # shape: (time, lat, lon)
features = xr.Dataset(
    {
        "VHM0_uncorr": ds_features["VHM0"],
        "WSPD": ds_features["WSPD"],
        "WDIR_cos": ds_features["WDIR_cos"],
        "WDIR_sin": ds_features["WDIR_sin"],
        "VMDR_cos": ds_features["VMDR_cos"],
        "VMDR_sin": ds_features["VMDR_sin"],
        "month_cos": ds_features["month_cos"],
        "month_sin": ds_features["month_sin"],
        "hour_cos": ds_features["hour_cos"],
        "hour_sin": ds_features["hour_sin"],
    }
).to_array(dim="channel")  # shape: (channel, time, latitude, longitude)

# Transpose to (time, latitude, longitude, channel)
features = features.transpose("time", "latitude", "longitude", "channel")
channels_number = features.sizes["channel"]

# Split sets

In [ ]:
train_features = features.sel(time=slice("2021-01-01", "2022-12-31"))
train_labels = labels.sel(time=slice("2021-01-01", "2022-12-31"))

val_features = xr.concat(
    [
        features.sel(time=slice("2023-01-01", "2023-01-31")),  # Winter
        features.sel(time=slice("2023-04-01", "2023-04-30")),  # Spring
        features.sel(time=slice("2023-07-01", "2023-07-31")),  # Summer
        features.sel(time=slice("2023-10-01", "2023-10-31")),  # Fall
    ],
    dim="time",
)

val_labels = xr.concat(
    [
        labels.sel(time=slice("2023-01-01", "2023-01-31")),
        labels.sel(time=slice("2023-04-01", "2023-04-30")),
        labels.sel(time=slice("2023-07-01", "2023-07-31")),
        labels.sel(time=slice("2023-10-01", "2023-10-31")),
    ],
    dim="time",
)

test_features = xr.concat(
    [
        features.sel(time=slice("2023-02-01", "2023-02-28")),  # Winter
        features.sel(time=slice("2023-05-01", "2023-05-31")),  # Spring
        features.sel(time=slice("2023-08-01", "2023-08-31")),  # Summer
        features.sel(time=slice("2023-11-01", "2023-11-30")),  # Fall
    ],
    dim="time",
)

test_labels = xr.concat(
    [
        labels.sel(time=slice("2023-02-01", "2023-02-28")),
        labels.sel(time=slice("2023-05-01", "2023-05-31")),
        labels.sel(time=slice("2023-08-01", "2023-08-31")),
        labels.sel(time=slice("2023-11-01", "2023-11-30")),
    ],
    dim="time",
)

#Scalers

In [ ]:
channel_names = [
    "VHM0",
    "WSPD",
    "WDIR_cos",
    "WDIR_sin",
    "VMDR_cos",
    "VMDR_sin",
    "month_cos",
    "month_sin",
    "hour_cos",
    "hour_sin",
]
scalers = {}

for i, name in enumerate(channel_names):
    # Compute mean and std lazily with Dask for channel i
    channel_data = train_features[..., i]
    mean_val = channel_data.mean().compute()
    std_val = channel_data.std().compute()
    print(f" Train set: Channel {name} - Mean: {mean_val}, Std: {std_val}")
    scaler = StandardScaler()
    scaler.mean_ = np.array([mean_val])
    scaler.scale_ = np.array([std_val])
    scalers[name] = scaler

label_scaler = StandardScaler()
train_labels_np = train_labels.compute().data.reshape(-1, 1)
label_scaler.fit(train_labels_np)


def scale_feature(data, scaler):
    original_shape = data.shape
    data_flat = data.reshape(-1, 1)
    data_scaled = scaler.transform(data_flat)
    return data_scaled.reshape(original_shape)


def scale_label(data, scaler):
    original_shape = data.shape
    data_flat = data.reshape(-1, 1)
    data_scaled = scaler.transform(data_flat)
    return data_scaled.reshape(original_shape)

In [ ]:
def data_generator(features, labels, feature_scalers, label_scaler, batch_size):
    num_samples = features.sizes["time"]
    for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)
        # Load the batch into memory
        batch_features = (
            features.isel(time=slice(start, end)).compute().data
        )  # shape: (batch, lat, lon, channels)
        batch_labels = (
            labels.isel(time=slice(start, end)).compute().data[..., None]
        )  # shape: (batch, lat, lon, 1)

        # Scaling
        for i, name in enumerate(channel_names):
            batch_features[..., i] = scale_feature(
                batch_features[..., i], feature_scalers[name]
            )
        batch_labels = scale_label(batch_labels, label_scaler)

        batch_features = np.nan_to_num(batch_features, nan=0.0)
        batch_labels = np.nan_to_num(batch_labels, nan=0.0)

        # Convert to numpy arrays
        yield batch_features.astype(np.float32), batch_labels.astype(np.float32)

In [ ]:
def create_dataset(
    features_subset, labels_subset, feature_scalers, label_scaler, batch_size
):
    return tf.data.Dataset.from_generator(
        lambda: data_generator(
            features_subset, labels_subset, feature_scalers, label_scaler, batch_size
        ),
        output_types=(tf.float32, tf.float32),
        output_shapes=(
            (
                None,
                features_subset.sizes["latitude"],
                features_subset.sizes["longitude"],
                features_subset.sizes["channel"],
            ),
            (
                None,
                labels_subset.sizes["latitude"],
                labels_subset.sizes["longitude"],
                1,
            ),
        ),
    )


BATCH_SIZE = 48

train_dataset = create_dataset(
    train_features, train_labels, scalers, label_scaler, BATCH_SIZE
)
val_dataset = create_dataset(
    val_features, val_labels, scalers, label_scaler, BATCH_SIZE
)
test_dataset = create_dataset(
    test_features, test_labels, scalers, label_scaler, BATCH_SIZE
)

train_dataset = (
    train_dataset.cache()
    .shuffle(buffer_size=100, reshuffle_each_iteration=True)
    .repeat()
    .prefetch(tf.data.AUTOTUNE)
)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

#Defining the Deep Learning Model

**Masking in the Loss Function**
Create a custom loss that ignores the NaN (land) locations.

In [ ]:
def masked_mse(y_true, y_pred, epsilon=1e-6):
    # Binary mask: 1 for valid values, 0 for land
    mask = tf.cast(tf.logical_not(tf.math.is_nan(y_true)), tf.float32)
    mask_sum = tf.reduce_sum(mask)

    y_true_clean = tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_true), y_true)
    y_pred_clean = tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_pred), y_pred)

    # Compute the squared error and apply the mask.
    se = tf.square(y_true_clean - y_pred_clean) * mask

    # If no valid pixels in batch, return 0.
    mse = tf.cond(
        tf.equal(mask_sum, 0.0),
        lambda: 0.0,
        lambda: tf.reduce_sum(se) / (mask_sum + epsilon),
    )
    return mse


def masked_mae(y_true, y_pred, epsilon=1e-6):
    # Mask: 1 for valid pixels, 0 for NaNs.
    mask = tf.cast(tf.logical_not(tf.math.is_nan(y_true)), tf.float32)
    mask_sum = tf.reduce_sum(mask)

    # Replace NaNs with zeros for the error computation.
    y_true_clean = tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_true), y_true)
    y_pred_clean = tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_pred), y_pred)

    # Compute the absolute error and apply the mask.
    abs_err = tf.abs(y_true_clean - y_pred_clean) * mask

    # Return 0 if no valid pixels, else the mean error.
    mae = tf.cond(
        tf.equal(mask_sum, 0.0),
        lambda: 0.0,
        lambda: tf.reduce_sum(abs_err) / (mask_sum + epsilon),
    )
    return mae

In [ ]:
class ReflectionPadding2D(tf.keras.layers.Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        super(ReflectionPadding2D, self).__init__(**kwargs)
        self.padding = padding

    def call(self, x):
        if isinstance(self.padding[0], (list, tuple)):
            pad_top, pad_bottom = self.padding[0]
            pad_left, pad_right = self.padding[1]
        else:
            pad_top = pad_bottom = self.padding[0]
            pad_left = pad_right = self.padding[1]
        return tf.pad(
            x,
            [[0, 0], [pad_top, pad_bottom], [pad_left, pad_right], [0, 0]],
            mode="REFLECT",
        )

In [ ]:
def unet_with_residual(input_shape):
    inputs = Input(shape=input_shape)
    padded_inputs = ReflectionPadding2D(padding=((2, 2), (1, 1)))(inputs)
    # Encoder
    c1 = Conv2D(32, (3, 3), padding="same", activation="relu")(padded_inputs)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), padding="same", activation="relu")(p1)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(128, (3, 3), padding="same", activation="relu")(p2)
    c3 = BatchNormalization()(c3)

    # Decoder
    u2 = UpSampling2D((2, 2))(c3)
    u2 = concatenate([u2, c2])
    c4 = Conv2D(64, (3, 3), padding="same", activation="relu")(u2)
    c4 = BatchNormalization()(c4)

    u1 = UpSampling2D((2, 2))(c4)
    u1 = concatenate([u1, c1])
    c5 = Conv2D(32, (3, 3), padding="same", activation="relu")(u1)
    c5 = BatchNormalization()(c5)

    # Output correction (same spatial shape, 1 channel)
    correction = Conv2D(1, (3, 3), padding="same", activation="linear")(c5)

    # Residual connection
    VHM0_raw = Lambda(lambda x: tf.expand_dims(x[..., 0], axis=-1))(padded_inputs)
    output_padded = Add()([VHM0_raw, correction])
    outputs = Cropping2D(cropping=((2, 2), (1, 1)))(
        output_padded
    )  # Removes extra padding
    model = Model(inputs, outputs)
    return model

In [ ]:
sample_batch_features, sample_batch_labels = next(
    data_generator(features, labels, scalers, label_scaler, batch_size=1)
)
print("Feature batch shape:", sample_batch_features.shape)
print("Label batch shape:", sample_batch_labels.shape)

In [ ]:
input_shape = (
    sample_batch_features.shape[1],
    sample_batch_features.shape[2],
    sample_batch_features.shape[3],
)
print(input_shape)
model = unet_with_residual(input_shape)

model.compile(optimizer="adam", loss=masked_mse, metrics=[masked_mae])
model.summary()

In [ ]:
train_time_steps = train_features.sizes["time"]

train_batches = train_time_steps // BATCH_SIZE
val_batches = val_features.sizes["time"] // BATCH_SIZE
test_batches = int(np.ceil(test_features.sizes["time"] / BATCH_SIZE))

In [ ]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=10, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True
)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    steps_per_epoch=train_batches,
    validation_steps=val_batches,
    callbacks=[lr_scheduler, early_stopping],
)

In [ ]:
model.save("/content/drive/MyDrive/saved_model_BUnet_02.keras")
model.save_weights("/content/drive/MyDrive/weights_bunet02.weights.h5")

 Polynomial model

In [ ]:
class PolynomialExpansion(Layer):
    def __init__(self, degree=2, **kwargs):
        super(PolynomialExpansion, self).__init__(**kwargs)
        self.degree = degree
        if self.degree != 2:
            raise NotImplementedError(
                "This implementation currently supports only degree=2."
            )

    def call(self, inputs):
        # inputs shape: (batch, height, width, channels)
        # For each pixel, we want to compute the original features and all quadratic terms.
        batch_size, H, W, C = tf.unstack(tf.shape(inputs))
        # Reshape spatial dimensions into one: (batch * H * W, C)
        flat_inputs = tf.reshape(inputs, [-1, C])
        # Compute the full outer product for each pixel: shape (batch*H*W, C, C)
        outer = tf.einsum("bi,bj->bij", flat_inputs, flat_inputs)
        # Flatten the quadratic terms: shape (batch*H*W, C*C)
        outer_flat = tf.reshape(outer, [-1, C * C])
        # Concatenate original features and quadratic features: shape (batch*H*W, C + C*C)
        expanded_flat = tf.concat([flat_inputs, outer_flat], axis=-1)
        # Reshape back to (batch, H, W, new_channels)
        new_channels = C + C * C
        expanded = tf.reshape(expanded_flat, [batch_size, H, W, new_channels])
        return expanded

    def compute_output_shape(self, input_shape):
        return (
            input_shape[0],
            input_shape[1],
            input_shape[2],
            input_shape[3] + input_shape[3] * input_shape[3],
        )


def polynomial_model(input_shape, degree=2):
    inputs = Input(shape=input_shape)

    if degree == 1:
        # If degree=1, the model is equivalent to a linear model.
        expanded = inputs
    elif degree == 2:
        expanded = PolynomialExpansion(degree=2)(inputs)
    else:
        raise NotImplementedError("Only degree=1 and degree=2 are implemented.")

    # The 1x1 convolution learns a linear combination of the expanded polynomial features.
    outputs = Conv2D(1, (1, 1), padding="same", activation="linear")(expanded)
    model = Model(inputs, outputs)
    return model


input_shape = (
    sample_batch_features.shape[1],
    sample_batch_features.shape[2],
    sample_batch_features.shape[3],
)
poly_model = polynomial_model(input_shape, degree=2)
poly_model.summary()


poly_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=masked_mse,
    metrics=[masked_mae, masked_rmse],
)

#Testing and Evaluating the Model

In [ ]:
def reverse_scaling(data, scaler):
    return data * scaler.scale_[0] + scaler.mean_[0]


def compute_masked_rmse(y_true, y_pred, epsilon=1e-6):
    mask = ~np.isnan(y_true)
    y_true_valid = y_true[mask]
    y_pred_valid = y_pred[mask]
    if y_true_valid.size == 0:
        return np.nan
    return np.sqrt(np.mean((y_true_valid - y_pred_valid) ** 2))


def compute_masked_mae(y_true, y_pred, epsilon=1e-6):
    mask = ~np.isnan(y_true)
    y_true_valid = y_true[mask]
    y_pred_valid = y_pred[mask]
    if y_true_valid.size == 0:
        return np.nan
    return np.mean(np.abs(y_true_valid - y_pred_valid))


def compute_masked_bias(y_true, y_pred):
    mask = ~np.isnan(y_true)
    y_true_valid = y_true[mask]
    y_pred_valid = y_pred[mask]
    if y_true_valid.size == 0:
        return np.nan
    return np.mean(y_pred_valid - y_true_valid)


def compute_masked_pearson(y_true, y_pred):
    mask = ~np.isnan(y_true)
    y_true_valid = y_true[mask]
    y_pred_valid = y_pred[mask]
    if y_true_valid.size == 0:
        return np.nan
    corr, _ = pearsonr(y_true_valid, y_pred_valid)
    return corr


def evaluate_model(model, test_dataset, steps, label_scaler):
    all_preds = []
    all_labels = []

    for i, (x, y) in enumerate(test_dataset.take(steps)):
        preds = model.predict(x)
        all_preds.append(preds)
        all_labels.append(y)

    # Concatenate batches along time axis
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Reverse the scaling
    all_preds_orig = reverse_scaling(all_preds, label_scaler)
    all_labels_orig = reverse_scaling(all_labels, label_scaler)

    # Compute metrics
    rmse = compute_masked_rmse(all_labels_orig, all_preds_orig)
    mae = compute_masked_mae(all_labels_orig, all_preds_orig)
    bias = compute_masked_bias(all_labels_orig, all_preds_orig)
    corr = compute_masked_pearson(all_labels_orig, all_preds_orig)

    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"Bias (Mean Error): {bias:.4f}")
    print(f"Pearson Correlation: {corr:.4f}")

    metrics = {"RMSE": rmse, "MAE": mae, "Bias": bias, "Pearson": corr}
    return all_preds_orig, all_labels_orig, metrics


def plot_evaluation_results(all_preds, all_labels):
    plt.figure(figsize=(8, 6))
    plt.scatter(all_labels.reshape(-1), all_preds.reshape(-1), alpha=0.2, color="blue")
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("Scatter Plot of True vs. Predicted Values")
    min_val = 0
    max_val = 15
    plt.plot([min_val, max_val], [min_val, max_val], "r--")
    plt.show()


def plot_metrics_table(metrics):
    table_data = [
        ["RMSE", f"{metrics['RMSE']:.4f}"],
        ["MAE", f"{metrics['MAE']:.4f}"],
        ["Bias", f"{metrics['Bias']:.4f}"],
        ["Pearson", f"{metrics['Pearson']:.4f}"],
    ]

    fig, ax = plt.subplots(figsize=(4, 2))
    ax.axis("tight")
    ax.axis("off")
    table = ax.table(
        cellText=table_data,
        colLabels=["Metric", "Value"],
        cellLoc="center",
        loc="center",
    )
    plt.title("Evaluation Metrics (Masked)")
    plt.show()


all_preds_orig, all_labels_orig, metrics = evaluate_model(
    model, test_dataset, steps=test_batches, label_scaler=label_scaler
)
# Plot
plot_evaluation_results(all_preds_orig, all_labels_orig)
plot_metrics_table(metrics)

In [ ]:
all_features_orig = []
all_labels_orig = []

for x, y in test_dataset:
    x_np = x.numpy()
    y_np = y.numpy()
    x_feature = x_np[..., 0]

    x_feature_orig = (
        scalers["VHM0"]
        .inverse_transform(x_feature.reshape(-1, 1))
        .reshape(x_feature.shape)
    )
    y_label_orig = reverse_scaling(y_np, label_scaler)

    all_features_orig.append(x_feature_orig)
    all_labels_orig.append(y_label_orig)

all_features_orig = np.concatenate(all_features_orig, axis=0)
all_labels_orig = np.concatenate(all_labels_orig, axis=0)

all_labels_orig = all_labels_orig[..., 0]


rmse = compute_masked_rmse(all_labels_orig, all_features_orig)
mae = compute_masked_mae(all_labels_orig, all_features_orig)
bias = compute_masked_bias(all_labels_orig, all_features_orig)
corr = compute_masked_pearson(all_labels_orig, all_features_orig)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"Bias (Mean Error): {bias:.4f}")
print(f"Pearson Correlation: {corr:.4f}")
metrics = {"RMSE": rmse, "MAE": mae, "Bias": bias, "Pearson": corr}


def plot_evaluation_results(all_preds, all_labels):
    plt.figure(figsize=(8, 6))
    plt.scatter(all_labels.reshape(-1), all_preds.reshape(-1), alpha=0.2, color="blue")
    plt.xlabel("True Values")
    plt.ylabel("Uncorrected Values")
    plt.title("Scatter Plot of True vs. Uncorrected Values")
    min_val = 0
    max_val = 15
    plt.plot([min_val, max_val], [min_val, max_val], "r--")
    plt.show()


def plot_metrics_table(metrics):
    table_data = [
        ["RMSE", f"{metrics['RMSE']:.4f}"],
        ["MAE", f"{metrics['MAE']:.4f}"],
        ["Bias", f"{metrics['Bias']:.4f}"],
        ["Pearson", f"{metrics['Pearson']:.4f}"],
    ]

    fig, ax = plt.subplots(figsize=(4, 2))
    ax.axis("tight")
    ax.axis("off")
    table = ax.table(
        cellText=table_data,
        colLabels=["Metric", "Value"],
        cellLoc="center",
        loc="center",
    )
    plt.title("Metrics of True vs Uncorrected values")
    plt.show()


plot_evaluation_results(all_features_orig, all_labels_orig)
plot_metrics_table(metrics)

NameError: name 'test_dataset' is not defined

In [ ]:
# Metrics for Predictions vs. Labels
rmse_map_pred = np.sqrt(np.mean((all_preds_orig - all_labels_orig) ** 2, axis=0))
mae_map_pred = np.mean(np.abs(all_preds_orig - all_labels_orig), axis=0)
bias_map_pred = np.mean(all_preds_orig - all_labels_orig, axis=0)
avg_label_map = np.mean(all_labels_orig, axis=0)
# Metrics for Features vs. Labels
rmse_map_feat = np.sqrt(np.mean((all_features_orig - all_labels_orig) ** 2, axis=0))
mae_map_feat = np.mean(np.abs(all_features_orig - all_labels_orig), axis=0)
bias_map_feat = np.mean(all_features_orig - all_labels_orig, axis=0)


H, W = avg_label_map.shape
land_mask = np.isnan(labels).all(dim="time")

# Convert from xarray DataArray to a plain numpy array (if needed):
land_mask = land_mask.data  # shape: (lat, lon)

# ---------------------------
# Prepare a Custom "jet" Colormap with Land as White
# ---------------------------
cmap = plt.get_cmap("jet").copy()
cmap.set_bad("white")  # Masked values will appear white

# Create masked arrays for all computed maps using the land mask
rmse_map_pred_masked = np.ma.array(rmse_map_pred, mask=land_mask)
mae_map_pred_masked = np.ma.array(mae_map_pred, mask=land_mask)
bias_map_pred_masked = np.ma.array(bias_map_pred, mask=land_mask)

rmse_map_feat_masked = np.ma.array(rmse_map_feat, mask=land_mask)
mae_map_feat_masked = np.ma.array(mae_map_feat, mask=land_mask)
bias_map_feat_masked = np.ma.array(bias_map_feat, mask=land_mask)


# Difference Maps
diff_rmse_map = rmse_map_feat - rmse_map_pred
diff_mae_map = mae_map_feat - mae_map_pred
diff_bias_map = bias_map_feat - bias_map_pred

diff_rmse_map_masked = np.ma.array(diff_rmse_map, mask=land_mask)
diff_mae_map_masked = np.ma.array(diff_mae_map, mask=land_mask)
diff_bias_map_masked = np.ma.array(diff_bias_map, mask=land_mask)
# Compute global min/max for each metric separately:
rmse_vmin = 0
rmse_vmax = 0.35

mae_vmin = 0
mae_vmax = 0.2

bias_vmin = -0.15
bias_vmax = 0.15


def plot_map(ax, data, title, mask, vmin, vmax):
    im = ax.imshow(data, cmap=cmap, origin="lower", vmin=vmin, vmax=vmax)
    ax.set_title(title)
    plt.colorbar(im, ax=ax)
    ax.contour(
        mask.astype(float), levels=[0.5], colors="black", linewidths=1, origin="lower"
    )


# Predictions vs. Labels
fig1, axes1 = plt.subplots(1, 3, figsize=(20, 6))
plot_map(
    axes1[0],
    rmse_map_pred_masked,
    "RMSE Map (Predictions vs. Labels)",
    land_mask,
    rmse_vmin,
    rmse_vmax,
)
plot_map(
    axes1[1],
    mae_map_pred_masked,
    "MAE Map (Predictions vs. Labels)",
    land_mask,
    mae_vmin,
    mae_vmax,
)
plot_map(
    axes1[2],
    bias_map_pred_masked,
    "Bias Map (Predictions vs. Labels)",
    land_mask,
    bias_vmin,
    bias_vmax,
)
plt.tight_layout()
plt.show()

#  Features vs. Labels
fig2, axes2 = plt.subplots(1, 3, figsize=(20, 6))
plot_map(
    axes2[0],
    rmse_map_feat_masked,
    "RMSE Map (Features vs. Labels)",
    land_mask,
    rmse_vmin,
    rmse_vmax,
)
plot_map(
    axes2[1],
    mae_map_feat_masked,
    "MAE Map (Features vs. Labels)",
    land_mask,
    mae_vmin,
    mae_vmax,
)
plot_map(
    axes2[2],
    bias_map_feat_masked,
    "Bias Map (Features vs. Labels)",
    land_mask,
    bias_vmin,
    bias_vmax,
)
plt.tight_layout()
plt.show()

#  Difference Maps
fig3, axes3 = plt.subplots(1, 3, figsize=(20, 6))
plot_map(
    axes3[0],
    diff_rmse_map_masked,
    "Difference RMSE Map (Features - Predictions)",
    land_mask,
    -0.05,
    0.04,
)
plot_map(
    axes3[1],
    diff_mae_map_masked,
    "Difference MAE Map (Features - Predictions)",
    land_mask,
    -0.05,
    0.04,
)
plot_map(
    axes3[2],
    diff_bias_map_masked,
    "Difference Bias Map (Features - Predictions)",
    land_mask,
    -0.09,
    0,
)
plt.tight_layout()
plt.show()

In [ ]:
def compute_metrics_for_bin(y_true, y_pred, bin_lower, bin_upper):
    mask = (y_true >= bin_lower) & (y_true < bin_upper) & (~np.isnan(y_true))
    y_true_bin = y_true[mask]
    y_pred_bin = y_pred[mask]
    sample_count = y_true_bin.size
    if sample_count == 0:
        return np.nan, np.nan, np.nan, 0
    rmse = np.sqrt(np.mean((y_true_bin - y_pred_bin) ** 2))
    mae = np.mean(np.abs(y_true_bin - y_pred_bin))
    bias = np.mean(y_pred_bin - y_true_bin)
    return rmse, mae, bias, sample_count


bin_edges = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
bin_labels = [f"{bin_edges[i]}-{bin_edges[i + 1]}" for i in range(len(bin_edges) - 1)]
n_bins = len(bin_edges) - 1


rmse_pred, mae_pred, bias_pred = [], [], []
rmse_feat, mae_feat, bias_feat = [], [], []
sample_counts = []

for i in range(n_bins):
    r_pred, m_pred, b_pred, count = compute_metrics_for_bin(
        all_labels_orig, all_preds_orig, bin_edges[i], bin_edges[i + 1]
    )
    r_feat, m_feat, b_feat, count_feat = compute_metrics_for_bin(
        all_labels_orig, all_features_orig, bin_edges[i], bin_edges[i + 1]
    )
    rmse_pred.append(r_pred)
    mae_pred.append(m_pred)
    bias_pred.append(b_pred)
    rmse_feat.append(r_feat)
    mae_feat.append(m_feat)
    bias_feat.append(b_feat)
    sample_counts.append(count)

bar_width = 0.35
x = np.arange(n_bins)

# RMSE Plot
plt.figure(figsize=(10, 6))
bars1 = plt.bar(
    x - bar_width / 2,
    rmse_feat,
    width=bar_width,
    label="Features vs. Labels",
    color="tab:orange",
)
bars2 = plt.bar(
    x + bar_width / 2,
    rmse_pred,
    width=bar_width,
    label="Predictions vs. Labels",
    color="tab:blue",
)
plt.xticks(x, bin_labels)
plt.xlabel("VHM0 Range (m)")
plt.ylabel("RMSE")
plt.title("RMSE for Each VHM0 Range")
plt.ylim(0, 1.1)
# sample size above each bin group
for i in range(n_bins):
    y_val = max(rmse_feat[i], rmse_pred[i])
    plt.text(
        x[i],
        y_val + 0.05 * y_val,
        f"n={sample_counts[i]}",
        ha="center",
        va="bottom",
        fontsize=10,
    )
plt.legend()
plt.show()

# MAE Plot
plt.figure(figsize=(10, 6))
bars1 = plt.bar(
    x - bar_width / 2,
    mae_feat,
    width=bar_width,
    label="Features vs. Labels",
    color="tab:orange",
)
bars2 = plt.bar(
    x + bar_width / 2,
    mae_pred,
    width=bar_width,
    label="Predictions vs. Labels",
    color="tab:blue",
)
plt.xticks(x, bin_labels)
plt.xlabel("VHM0 Range (m)")
plt.ylabel("MAE")
plt.title("MAE for Each VHM0 Range")
plt.ylim(0, 1.1)
for i in range(n_bins):
    y_val = max(mae_feat[i], mae_pred[i])
    plt.text(
        x[i],
        y_val + 0.05 * y_val,
        f"n={sample_counts[i]}",
        ha="center",
        va="bottom",
        fontsize=10,
    )
plt.legend()
plt.show()

# Bias Plot
plt.figure(figsize=(10, 6))
bars1 = plt.bar(
    x - bar_width / 2,
    bias_feat,
    width=bar_width,
    label="Features vs. Labels",
    color="tab:orange",
)
bars2 = plt.bar(
    x + bar_width / 2,
    bias_pred,
    width=bar_width,
    label="Predictions vs. Labels",
    color="tab:blue",
)
plt.xticks(x, bin_labels)
plt.xlabel("VHM0 Range (m)")
plt.ylabel("Bias")
plt.title("Bias for Each VHM0 Range")
plt.ylim(-1.1, 0.2)
for i in range(n_bins):
    y_val = max(bias_feat[i], bias_pred[i])
    # Adjust offset differently if bias is negative.
    offset = 0.05 * abs(y_val) if y_val != 0 else 0.05
    plt.text(
        x[i],
        y_val + offset,
        f"n={sample_counts[i]}",
        ha="center",
        va="bottom",
        fontsize=10,
    )
plt.legend()
plt.show()

NameError: name 'all_preds_orig' is not defined